In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, Subset
import torch
from torch import nn as nn
from sklearn.neural_network import MLPClassifier

In [2]:
#df_data = pd.read_csv('bert_embeddings_small.csv')
df_data_neg = pd.read_csv('bert_embeddings_neg.csv')
df_data_pos = pd.read_csv('bert_embeddings_pos.csv')
data_neg = df_data_neg.values
data_pos = df_data_pos.values
df_data_complete = pd.concat([df_data_neg, df_data_pos])
orig_data_complete = df_data_complete.values

In [3]:
#orig_data = df_data.values

In [4]:
'''n_components = 10
pca = PCA(n_components=n_components)
data_pca = pca.fit_transform(orig_data)'''

'n_components = 10\npca = PCA(n_components=n_components)\ndata_pca = pca.fit_transform(orig_data)'

In [5]:
'''data_reconstructed = pca.inverse_transform(data_pca)'''

'data_reconstructed = pca.inverse_transform(data_pca)'

In [6]:
'''def mse(x, y):
    return np.mean((x - y) ** 2)

def plot_array(data):
    plt.figure(figsize=(10, 6))  # Set the figure size
    plt.plot(data)  # Plot the data
    plt.title('Data Plot')  # Set the title of the plot
    plt.xlabel('Index')  # Set the x-axis label
    plt.ylabel('Value')  # Set the y-axis label
    plt.grid(True)  # Show grid
    plt.show()  # Display the plot
    
total_mse = 0
for i in tqdm(range(len(orig_data))):
    total_mse += mse(orig_data[i], data_reconstructed[i])
total_mse'''

"def mse(x, y):\n    return np.mean((x - y) ** 2)\n\ndef plot_array(data):\n    plt.figure(figsize=(10, 6))  # Set the figure size\n    plt.plot(data)  # Plot the data\n    plt.title('Data Plot')  # Set the title of the plot\n    plt.xlabel('Index')  # Set the x-axis label\n    plt.ylabel('Value')  # Set the y-axis label\n    plt.grid(True)  # Show grid\n    plt.show()  # Display the plot\n    \ntotal_mse = 0\nfor i in tqdm(range(len(orig_data))):\n    total_mse += mse(orig_data[i], data_reconstructed[i])\ntotal_mse"

In [7]:
'''errors = []
for i in tqdm(range(100)):
    pca = PCA(n_components=i+5)
    data_pca = pca.fit_transform(orig_data)
    data_reconstructed = pca.inverse_transform(data_pca)
    total_mse = 0
    for i in (range(len(orig_data))):
        total_mse += mse(orig_data[i], data_reconstructed[i])
    errors.append(total_mse)'''

'errors = []\nfor i in tqdm(range(100)):\n    pca = PCA(n_components=i+5)\n    data_pca = pca.fit_transform(orig_data)\n    data_reconstructed = pca.inverse_transform(data_pca)\n    total_mse = 0\n    for i in (range(len(orig_data))):\n        total_mse += mse(orig_data[i], data_reconstructed[i])\n    errors.append(total_mse)'

In [8]:
'''scaled_errors = np.array(errors) / len(orig_data)
plot_array(scaled_errors)'''

'scaled_errors = np.array(errors) / len(orig_data)\nplot_array(scaled_errors)'

In [9]:
final_PCA = PCA(n_components=100)
final_PCA.fit(orig_data_complete)
low_dim_neg = final_PCA.transform(data_neg)
low_dim_pos = final_PCA.transform(data_pos)
reconstr_neg = final_PCA.inverse_transform(low_dim_neg)
reconstr_pos = final_PCA.inverse_transform(low_dim_pos)

In [10]:
def append_number_to_rows(arr, num):
    num_column = np.full((arr.shape[0], 1), num)
    return np.hstack((arr, num_column))

low_dim_neg_lbl = append_number_to_rows(low_dim_neg, 0)[:, :]
low_dim_pos_lbl = append_number_to_rows(low_dim_pos, 1)[:, :]
reconstr_neg_lbl = append_number_to_rows(reconstr_neg, 0)[:, :]
reconstr_pos_lbl = append_number_to_rows(reconstr_pos, 1)[:, :]
low_dim_complete_lbl = np.vstack((low_dim_neg_lbl, low_dim_pos_lbl))
reconstr_complete_lbl = np.vstack((reconstr_neg_lbl, reconstr_pos_lbl))


In [11]:
reconstr_complete_lbl.shape

(2500000, 769)

In [12]:
class NumpyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx, :-1], dtype=torch.float), torch.tensor(self.data[idx, -1], dtype=torch.float)

def create_dataloaders_from_df(dataframe, test_size=0.2, batch_size=32, shuffle=True):
    # Split the DataFrame into training and testing sets
    train_df, test_df = train_test_split(dataframe, test_size=test_size)
    
    # Create datasets
    train_dataset = NumpyDataset(train_df)
    test_dataset = NumpyDataset(test_df)
    
    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_dataloader, test_dataloader

class Classifier(nn.Module):
    def __init__(self, vec_dim, hidden_dim1, hidden_dim2, dropout_rate=0.1):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(vec_dim, hidden_dim1)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(hidden_dim2, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        #x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        #x = self.dropout2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    
def train_model(model, train_loader, test_loader, n_epochs, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer=torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn=nn.BCEWithLogitsLoss()
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader):
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  
            outputs = model(inputs) 
            loss = loss_fn(outputs.squeeze(), labels)  
            loss.backward() 
            optimizer.step()  
            total_loss += loss.item()
        print(f'Loss: {total_loss:.2f}')
        print(f'Testing accuracy: {evaluate_model(model, test_loader)}')
        
        
def evaluate_model(model, test_loader):
    model.eval() 
    correct = 0
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    with torch.no_grad():  
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            predicted = (outputs.squeeze() >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    return accuracy

In [13]:
train_loader_low_dim, test_loader_low_dim = create_dataloaders_from_df(low_dim_complete_lbl)
train_loader_reconstr, test_loader_reconstr = create_dataloaders_from_df(reconstr_complete_lbl)

model_low_dim = Classifier(100, 20, 10)
model_reconstr = Classifier(768, 500, 100)

In [14]:
train_model(model_low_dim, train_loader_low_dim, test_loader_low_dim, n_epochs=100, lr=0.0001)

100%|████████████████████████████████████████████████████████████████████████████| 62500/62500 [03:56<00:00, 264.12it/s]


Loss: 37558.22
Testing accuracy: 0.803476


100%|████████████████████████████████████████████████████████████████████████████| 62500/62500 [03:56<00:00, 264.41it/s]


Loss: 37090.31


KeyboardInterrupt: 

In [ ]:
model_save_path = 'aaaaamodel_low_dim.pth'
torch.save(model_low_dim.state_dict(), model_save_path)
print(f'Model saved to {model_save_path}')

Model saved to aaaaamodel_low_dim.pth


In [ ]:
train_model(model_reconstr, train_loader_reconstr, test_loader_reconstr, n_epochs=100)

  0%|                                                                                           | 0/500 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 313.79it/s]


Loss: 303.93
Training accuracy: 0.78125, Testing accuracy: 0.7735


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 312.07it/s]


Loss: 299.69
Training accuracy: 0.7869375, Testing accuracy: 0.7775


...

Loss: 285.63
Training accuracy: 0.859125, Testing accuracy: 0.80825


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 300.27it/s]


Loss: 284.39
Training accuracy: 0.8590625, Testing accuracy: 0.79775


 91%|████████████████████████████████████████████████████████████████████████▉       | 456/500 [00:01<00:00, 303.40it/s]


KeyboardInterrupt: 